# Refine an Existing LUT (.cube) Using a Second LUT (.cube) and Output a Smoothed New LUT file
This notebook allows you to refine an existing LUT curve from a .cube file using a second .cube file.  The workflow would normally be:
1. Run testing procedures and obtain the first LUT .cube file
1. Apply the first LUT .cube file to the same test image (step wedge) to make a new negative and print it
1. Obtain the second LUT .cube file from the second print
1. Refine/correct the first LUT .cube using the second LUT .cube file in this notebook

NOTE: It is assumed that all LUT .cube files are obtained from Easy Digital Negatives or that the .cube files were generated by the EDN_CSV_To_Smooth_Cube_File notebook (which uses Easy Digital Negatives CSV files).

### IMPORTANT
**Both .cube files must contain the same number of entries.  If one file has 256 entries then both files must have 256 entries.  You cannot use a 21-step .cube file in conjunction with a 256-step .cube file. If you try this, you will encounter errors.**
### Output .cube File
The output of this notebook is a new .cube file that contains the smoothed, refined LUT curve.  You could further refine that .cube by using it as the first LUT .cube file and run through these procedures again.  At some point you'll just be chasing perfection that cannot be attained, so it is uncommon to run through more than two iterations.  One refinement iteration may even be sufficient.

The output .cube file will use the name you provide (when prompted) and will be saved in the same folder as the first .cube file.
### How To Use This Notebook
The code in this notebook is organized into cells.  Each cell can be run independently, providing all prerequisite imports have been done and all variables exist.  However, **it is recommended that you run all code cells in sequence.**  

The exceptions to this normal operation are that you may wish to run the [Refine the First Curve Using the Second Curve](#Refine-the-First-Curve-Using-the-Second-Curve) or [Smooth the New Curve and Plot the Results](#Smooth-the-New-Curve-and-Plot-the-Results) cells more than once, providing different refinement or smoothing values until you are satisfied with the results.  Once you are happy with the resulting curve, you can then move on to the [Save the New Curve to a .cube File](#Save-the-New-Curve-to-a-.cube-File) cell to save the new .cube file.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter import filedialog
from tkinter import simpledialog
from scipy.interpolate import UnivariateSpline

In [ ]:
def read_cube_to_list(in_file):
    cube_values = []
    with open(in_file, 'r') as cf:
        for line in cf:
            if line.startswith("#"):
                continue
            if line.startswith("TITLE"):
                continue
            if line.startswith("LUT"):
                continue
            if line.startswith("DOMAIN"):
                continue
            if line:
                cube_values.append(float(line.split(" ")[0]))
    return cube_values

# Get the .cube Files to Process

In [ ]:
root = Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()
first_cube_file = filedialog.askopenfilename(title="First .cube file (to refine)", filetypes=[("LUT files", "*.cube")])
second_cube_file = filedialog.askopenfilename(title="Second (correcting) .cube file", filetypes=[("LUT files", "*.cube")])
min_val = 0.0
max_val = 1.0
first_cube_values = read_cube_to_list(first_cube_file)
second_cube_values = read_cube_to_list(second_cube_file)
cube_filename = simpledialog.askstring("Output .cube File Name", "Enter the name for the output .cube file (no .cube extension)")
cube_filename = cube_filename.replace(" ", "_")
print("Refining: {}".format(first_cube_file))
print("Using: {}".format(second_cube_file))
print("To create {}.cube".format(cube_filename))

# Refine the First Curve Using the Second Curve
When combining the first and second curves as-is, it is possible to overshoot the correction to get to the desired linear response.  To combat this, you will be prompted to provide a refinement factor.  If you provide a value of 2.0, that means that the correction applied to the first curve will be reduced by half the value it would have been with no refinement factor.  If you provide a value of 3.0, then it would be 1/3.  

The larger this number, the smaller the affect of the refinement.  Values between 2.0 and 4.0 are suggested.  

If you wish to not apply a refinement factor, provide a value of 1.0

**If, after running this and the smoothing cells, you do not like the resulting smoothed curve, you may re-run this cell as many times as needed to narrow down the refinement factor you like best.**

In [ ]:
value_steps = []
refine_factor = simpledialog.askfloat(title="Curve Refinement Factor", prompt="Please enter a compensation factor.\nExample: A value of 2.0 will cut the refinement correction in half", initialvalue=2.5)
if refine_factor == None or refine_factor <= 1.0:
    refine_factor = 1.0
print("Using a refinement factor of {}".format(refine_factor))
num_steps = len(first_cube_values)
step_increment = 1.0 / (num_steps - 1)
for i in range(num_steps):
    value_steps.append(round(min_val + (i * step_increment), 4))
linear_line = pd.Series(value_steps, index=value_steps)
first_curve_line = pd.Series(first_cube_values, index=value_steps)
second_curve_line = pd.Series(second_cube_values, index=value_steps)
cube_df = pd.DataFrame({'Linear':value_steps, 'First_Curve':first_curve_line, 'Second_Curve':second_curve_line}, index=value_steps)
cube_df['Refined_Curve'] = cube_df['First_Curve'] + ((cube_df['Second_Curve'] - cube_df['Linear']) / refine_factor)
print("Showing first 10 rows of data frame")
cube_df.head(10)

# Smooth the New Curve and Plot the Results
The following cell smooths the refined curve using a Univariate Spline with a smoothing factor.  You will be prompted to specify the smoothing factor.  Common values fall within the 0.1 to 0.001 range, and the larger this number, the smoother the resulting curve.  

If you do not wish to have any smoothing applied, enter 0.  

**If, after running this cell, you do not like the resulting smoothed curve, you may re-run this cell as many times as needed to narrow down the smoothing factor you like best.**

In [ ]:
smoothing_factor_value = simpledialog.askfloat(title="Smoothing Factor", prompt="Enter a smoothing factor. This will probably be a very small number less than 1.\nBigger numbers mean a smoother result.\nIf you enter 0, no further smoothing will be performed.", initialvalue=0.005)
if smoothing_factor_value == None:
    smoothing_factor_value = 0.005
curve_spline = UnivariateSpline(value_steps, cube_df['Refined_Curve'], k=4)
curve_spline.set_smoothing_factor(smoothing_factor_value)
if smoothing_factor_value > 0.0:
    cube_df['Refined_Smoothed_Curve'] = curve_spline(cube_df['Linear'])
plt.close("all")
first_file = os.path.basename(first_cube_file)
second_file = os.path.basename(second_cube_file)
title_text = "Curve Refinement\n\nRefinement Factor: {}\nSmoothing Factor: {}\n\n{}\nrefined using\n{}.cube".format(refine_factor, smoothing_factor_value, first_file, second_file)
cube_plot = cube_df.plot(grid=True, title=title_text, figsize=(7,7))
cube_plot.set_xlabel("Input Value")
cube_plot.set_ylabel("Output Value")
plt.show()

# Save the New Curve to a .cube File

In [ ]:
cube_filename = cube_filename.replace("plus", "minus")
out_cube = os.path.join(os.path.dirname(first_cube_file), "{}.cube".format(cube_filename))
with open(out_cube, 'w') as cube:
    cube.write("#Created by: Easy Digital Negatives plus Jupyter\nTITLE  \"{}\"\nLUT_1D_SIZE {}\nDOMAIN_MIN 0.0 0.0 0.0\nDOMAIN_MAX 1.0 1.0 1.0\n#LUT data points\n".format(cube_filename, num_steps))
    line_template = "{:<08} {:<08} {:<08}\n"
    for i, row in cube_df.iterrows():
        if smoothing_factor_value == 0.0:
            val = round(row['Refined_Curve'], 6)
        else:
            val = round(row['Refined_Smoothed_Curve'], 6)
        if val < 0:
            val = 0.0
        elif val > 1.0:
            val = 1.0
        cube.write(line_template.format(val, val, val))
    cube.write("#END data\n")
print("Saved {}".format(out_cube))